In [12]:
import firebase_admin
from firebase_admin import firestore
import os
import pandas as pd
from pathlib import Path
import sys
sys.path.append(os.path.dirname(os.getcwd() ))

import visualize


In [3]:
cred = firebase_admin.credentials.Certificate('key.json')
default_app = firebase_admin.initialize_app(cred)
db = firestore.client()
instructions_ref = db.collection('instructions')
verify_ref = db.collection('verification')


In [4]:
list_instruction = list(instructions_ref.get())
list_verification = list(verify_ref.get())
len(list_instruction), len(list_verification)


(3212, 3421)

In [48]:
instructions_ref_sandbox = db.collection('instructions_sandbox')
verify_ref_sandbox = db.collection('verification_sandbox')

list_instruction_sandbox = list(instructions_ref_sandbox.get())
list_verification_sandbox = list(verify_ref_sandbox.get())
len(list_instruction_sandbox), len(list_verification_sandbox)

(40, 75)

In [9]:
# Qualified workers
current_dir = os.getcwd()
parent_dir = Path(current_dir).parent.absolute()
path_qualified = os.path.join(parent_dir, "qualified_workers.csv")

qualified_work_list = pd.read_csv(path_qualified, header=None)[0].tolist()

qualified_work_list

['1',
 'AY7WPVKHVNBLG',
 'A1198W1SPF1R4',
 'A3QAL3X23LHJRV',
 'A3GWRDHAURRNK6',
 'A2DDPSXH2X96RF',
 'ALYR5CI2SM2JC',
 'A320QA9HJFUOZO',
 'APRZ7BR8C0ZMQ',
 'A3862RIFFUV141',
 'A1AKL5YH9NLD2V',
 'A2WX434EAQOE29',
 'AYIFHDQSXQJ6B']

In [19]:
# Set qualified instructions as RVS_excellent  - UPDATES DATA!!!
qualified_instructions = [
  e.to_dict()['key'] for e in list_instruction if (
    e.to_dict()['work_id'] in qualified_work_list) ]

for id in qualified_instructions:
  instructions_ref.document(id).update(
    {'review': 'RVS_excellent'})


In [ ]:
# Split instructions to train, dev, test sets.
import json

instruction_dir = "/mnt/hackney/human_data"

def save_split(split_name, split_set):

  split_path = os.path.join(instruction_dir, split_name)
  if os.path.exists(split_path):
    os.remove(split_path)
  with open(split_path, 'a') as outfile:
    for sample in split_set:
      json.dump(sample, outfile, default=lambda o: o.__dict__)
      outfile.write('\n')
      outfile.flush()

  print (f"Size of {split_name} set:{len(split_set)} saved to: {split_path}")


# Save set splits

dev_ids = ['X']
test_ids = ['X']


test_set =  [
  x.to_dict() for x in list_instruction if 'Manhattan' in x.to_dict()['region'] and x.to_dict()['work_id'] in test_ids]

save_split("test.json", test_set)

dev_set =  [
  x.to_dict() for x in list_instruction if 'Manhattan' in x.to_dict()['region'] and x.to_dict()['work_id'] in dev_ids]

save_split("dev.json", dev_set)

train_set = [
  x.to_dict() for x in list_instruction if 'Manhattan' in x.to_dict()['region'] and x.to_dict()['work_id'] not in dev_ids+test_ids]

save_split("train.json", train_set)


In [7]:
# check the new workers ID
df_instruction = pd.DataFrame(
    [
        x.to_dict() for x in list_instruction if 'Manhattan' in x.to_dict()['region'] and  'date_start' in x.to_dict() and '2022-11-01' <x.to_dict()['date_start'] ])

df_verification = pd.DataFrame(
        [x.to_dict() for x in list_verification] )


df = df_instruction.merge(df_verification, left_on='key', right_on='key_instruction')

work_ids = list(set(df['work_id_x'].to_list()))


for work_id in work_ids:
    if work_id in qualified_work_list:
        continue
    print (f"--------{work_id}--------")
    print (df[df['work_id_x']==work_id][['date_start_x', 'review', 'hit_id_x','dist_m']])


NameError: name 'qualified_work_list' is not defined

In [21]:
# Check the stack of RVS_excellent
all_data = pd.DataFrame([x.to_dict() for x in list_instruction ])
x = all_data[all_data['review']=='RVS_excellent' ]
x[x['verified_n']==0].shape

(0, 17)

In [28]:
# merge instructions with verification
df_instruction = pd.DataFrame([x.to_dict() for x in list_instruction])
df_ver = pd.DataFrame([x.to_dict() for x in list_verification ])

instruction_ver = df_instruction.merge(df_ver, left_on='key', right_on='key_instruction')


,rvs_goal_point_x,content,rvs_path_x,rvs_sample_number_x,key_x,review,verified_n,work_id_x,valid,region,...,key_instruction,task_y,work_id_y,assignmentId_y,hit_id_y,date_start_y,dist_m,rvs_path_y,date_finish_y,rvs_start_point_y
0,POINT (-73.99358176046474 40.74360156225971),Meet me at the church on West 23rd street. It ...,/app_instructor/data/manhattan_samples_v13.gpkg,1,1,RVS_excellent,15,1,True,Manhattan,...,1,1,A1198W1SPF1R4,3YMU66OBIN9BHRKVGQR0QS9OCSWHGI,3KLL7H3EGD2RD4CWFJLK2AH7U15HVR,2022-01-12 08:43:55.967628,4521.0,/app_instructor/data/manhattan_samples_v13.gpkg,2022-01-12 08:43:58.627970,POINT (-73.9971646 40.7475065)
1,POINT (-73.99358176046474 40.74360156225971),Meet me at the church on West 23rd street. It ...,/app_instructor/data/manhattan_samples_v13.gpkg,1,1,RVS_excellent,15,1,True,Manhattan,...,1,1,A21LONLNBOB8Q,3C2NJ6JBKAIDCTYZK91OT8CJO4K2NF,31ANT7FQN83AFAXH9S37YCN8HGF5HQ,2022-01-17 19:17:57.547530,1953.0,/app_instructor/data/manhattan_samples_v13.gpkg,2022-01-17 19:24:31.589128,POINT (-73.9971646 40.7475065)
2,POINT (-73.99358176046474 40.74360156225971),Meet me at the church on West 23rd street. It ...,/app_instructor/data/manhattan_samples_v13.gpkg,1,1,RVS_excellent,15,1,True,Manhattan,...,1,1,A2BQAB1BKV01YA,3LEIZ60CDJ0IT2XH86RXYIR65UB9ZH,379OL9DBSSFF2SPIMIKCWSNDTFF9Y2,2022-01-27 14:49:31.770290,533.0,/app_instructor/data/manhattan_samples_v13.gpkg,2022-01-27 14:51:18.836809,POINT (-73.9971646 40.7475065)
3,POINT (-73.99358176046474 40.74360156225971),Meet me at the church on West 23rd street. It ...,/app_instructor/data/manhattan_samples_v13.gpkg,1,1,RVS_excellent,15,1,True,Manhattan,...,1,1,A2GYGCLTMIGDJT,3XIQGXAUMC9PAO99SR5SN1KP3W4X7W,3L1EFR8WWT69218NHSSO77GXN8AF94,2022-02-01 04:34:03.817975,16.0,/app_instructor/data/manhattan_samples_v13.gpkg,2022-02-01 04:36:25.492434,POINT (-73.9971646 40.7475065)
4,POINT (-73.99358176046474 40.74360156225971),Meet me at the church on West 23rd street. It ...,/app_instructor/data/manhattan_samples_v13.gpkg,1,1,RVS_excellent,15,1,True,Manhattan,...,1,1,A2MOKIEQZ0OF2M,3G0WWMR1UVLU4C0KMKRPPQAM1IENQ7,36D1BWBEHN248RF5I6HMSASSDEO2M3,2021-12-23 14:23:33.423337,NaN,/app_instructor/data/manhattan_samples_v13.gpkg,2021-12-23 14:23:35.736300,POINT (-73.9971646 40.7475065)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3326,POINT (-73.9885442 40.7348845),Let’s meet at the restaurant.(Note: The goal m...,/app_instructor/data/manhattan_samples_v39.gpkg,270,AYIFHDQSXQJ6B3RBI0I35XE4TNQ0259J25FKRXA8Y3R2,NaN,1,AYIFHDQSXQJ6B,False,Manhattan,...,AYIFHDQSXQJ6B3RBI0I35XE4TNQ0259J25FKRXA8Y3R2,1,A3862RIFFUV141,31HQ4X3T3SBGTSCC2B0F7YMZMORLSL,3SBX2M1TKDO5WKSKED7R2Y9VD354Q2,2022-12-13 20:34:55.179855,2534.0,/app_instructor/data/manhattan_samples_v39.gpkg,2022-12-13 20:35:01.468384,POINT (-73.9872059 40.7497898)
3327,POINT (-73.990281 40.7797955),Meet me at Pier I right next to the bench. You...,/app_instructor/data/manhattan_samples_v45.gpkg,369,AYIFHDQSXQJ6B3RSBJ6YZECRC9OCTJHB64XBMNRROFV2,NaN,1,AYIFHDQSXQJ6B,True,Manhattan,...,AYIFHDQSXQJ6B3RSBJ6YZECRC9OCTJHB64XBMNRROFV2,1,A3862RIFFUV141,3OS46CRSLF0E2ZQ992ZIT2O9VUXV6N,3GVPRXWRPHVPY9MK330PW82JDV27IA,2022-12-13 00:20:33.714488,5.0,/app_instructor/data/manhattan_samples_v45.gpkg,2022-12-13 00:21:52.341207,POINT (-73.9801897 40.7729501)
3328,POINT (-74.01393078747282 40.706860794426696),Let’s meet up at the garden. Head straight wes...,/app_instructor/data/manhattan_samples_v43.gpkg,57,AYIFHDQSXQJ6B3ZC62PVYDHBG3JH6K24OTH2M30SXX02,RVS_excellent,1,AYIFHDQSXQJ6B,True,Manhattan,...,AYIFHDQSXQJ6B3ZC62PVYDHBG3JH6K24OTH2M30SXX02,1,A2WX434EAQOE29,3PB5A5BD0V7EO2EJ3H5OZFLV3PK7GY,3J94SKDEKIQT3USFNWKH7R8AUKZ5DR,2022-12-11 13:30:36.022694,9.0,/app_instructor/data/manhattan_samples_v43.gpkg,2022-12-11 13:35:32.122607,POINT (-74.002079 40.706986)
3329,POINT (-73.991226 40.7539106),We're going to meet at the bar on West 37th St...,/app_instructor/data/manhattan_samples_v13.gpkg,20,AYW62R027PUT131D0ZWOD0A0WAG326GN30V4DMQT0A

In [46]:
# Valid instruction
dist_threshold = 100


manhattan_instruction_ver = instruction_ver[instruction_ver['region']=='Manhattan']
pittsburgh_instruction_ver = instruction_ver[instruction_ver['region']=='Pittsburgh']


all_valid_instructions = instruction_ver[instruction_ver['dist_m']<dist_threshold]
manhattan_valid_instructions = all_valid_instructions[all_valid_instructions['region']=='Manhattan']
pittsburgh_valid_instructions = all_valid_instructions[all_valid_instructions['region']=='Pittsburgh']


valid_instructions = all_valid_instructions.groupby('key_instruction').min()
all_instructions = instruction_ver.groupby('key_x').min()
print (f"All valid for theshold {dist_threshold}: {round(100*valid_instructions.shape[0]/all_instructions.shape[0], 3)}" )


manhattan_valid_instructions = manhattan_valid_instructions.groupby('key_instruction').min()
manhattan_instructions = manhattan_instruction_ver.groupby('key_x').min()
print (f"Manhattan valid for theshold {dist_threshold}: {round(100*manhattan_valid_instructions.shape[0]/manhattan_instructions.shape[0], 3)}" )

pittsburgh_valid_instructions = pittsburgh_valid_instructions.groupby('key_instruction').min()
pittsburgh_instructions = pittsburgh_instruction_ver.groupby('key_x').min()
print (f"Pittsburgh valid for theshold {dist_threshold}: {round(100*pittsburgh_valid_instructions.shape[0]/pittsburgh_instructions.shape[0], 3)}" )




All valid for theshold 100: 74.175
Manhattan valid for theshold 100: 73.852
Pittsburgh valid for theshold 100: 74.766


In [56]:
#Update valid instructions
valid_instructions_tmp = [
  e.to_dict()['key'] for e in list_instruction if (
   e.to_dict()['key'] in all_valid_instructions['key_x'].tolist()) ]

len(valid_instructions_tmp)
for id in valid_instructions_tmp:
  instructions_ref.document(id).update(
    {'valid': True})

In [60]:
#How much was not verified at all
df_instruction[df_instruction['verified_n']==0].shape[0]

570

In [42]:
# Check number of completed assignments per HIT

hit_id = "X"
assignments = [
  e.to_dict()['key'] for e in list_instruction if (
   e.to_dict()['hit_id']==hit_id) ]

print(f"number of assignments for hit id {hit_id}: {len(assignments)}")


number of assignments for hit id X: 0


In [16]:
# visualize geosample with instruction

sample = 0
instruction = df_instruction.iloc[sample]
path_geodata = instruction['rvs_path']
final_path_geodata = os.path.join(parent_dir, path_geodata.replace("/app_instructor/", ""))
map_osm, _, _, _, _ = visualize.get_maps_and_instructions(final_path_geodata)[0]
print (instruction['content'])
map_osm


Meet me at the jewelry shop south of you on Canal Street. The jewelry shop is west of a church and north of a prison.

